# Predict
Predict gaze on normalized images

## Import libraries

In [1]:
import os, sys

import numpy as np
import pandas as pd
import tensorflow as tf

sys.path.append(os.path.abspath(os.path.join('..', '..')))
from Models import CNNResNetSWAttention
from Utils.LoadingUtils import read_images_separate

## Configuration

### Directories

In [2]:
model_path = '../../Models/rn_sw_attention__tf_model'
base_path = '../../Datasets/Study_2/Participants'
data_directory = '../../Datasets/Study_2/Meta.tsv'
normalised_directory = 'Normalised'

## Load model

In [3]:
def load_model(model_path, bn=True, first_dense_units=256, fc_layer_units=None, lr=None):
    if fc_layer_units is None:
        fc_layer_units = [256, 512]
    if lr is None:
        lr = [1e-5, 1e-4]

    cnn_model = CNNResNetSWAttention.create_resnet18_sw__attention(input_shape=(60, 36, 1),
                                                                   bn=bn,
                                                                   first_dense_units=first_dense_units,
                                                                   fc_layer_units=fc_layer_units,
                                                                   debug=False)

    cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    cnn_model.load_weights(model_path)

    return cnn_model

In [4]:
cnn_model = load_model(model_path,
                       bn=False,
                       first_dense_units=512,
                       fc_layer_units=[2048, 1024],
                       lr=[5e-6, 2.5e-4])

## Prediction

### Load meta

In [5]:
df = pd.read_csv(data_directory, header=0, delimiter='\t')

### Load images

In [6]:
def relative_path_to_absolute(row, right_eye=True):
    if right_eye is True:
        return f'{base_path}/{row["Participant"]}/{row["Case"]}/{normalised_directory}/{row["Right_eye_normalized"]}'
    else:
        return f'{base_path}/{row["Participant"]}/{row["Case"]}/{normalised_directory}/{row["Left_eye_normalized"]}'


df_not_null = df.loc[df['Right_eye_normalized'].notnull(), :]
right_eyes = df_not_null.apply(lambda row: relative_path_to_absolute(row, right_eye=True), axis=1)
left_eyes = df_not_null.apply(lambda row: relative_path_to_absolute(row, right_eye=False), axis=1)


def read_img(right, left):
    images, _ = read_images_separate(right, left, None, efficientnet=True)
    return images,


ds_normalized = tf.data.Dataset.from_tensor_slices((right_eyes, left_eyes))
ds_normalized = ds_normalized \
    .map(lambda right_eye_img, left_eye_img: read_img(right_eye_img, left_eye_img), num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(32) \
    .cache() \
    .prefetch(tf.data.AUTOTUNE)

### Execute prediction

In [7]:
ds_normalized_predicted = cnn_model.predict(ds_normalized, batch_size=32)

152/152 [==============================] - 28s 133ms/step


### Add predicted coordinates to meta dataframe

In [8]:
# Add to partial dataframe
ds_normalized_predicted = ds_normalized_predicted.transpose()
df_not_null['Rage-Net_X'] = np.multiply(ds_normalized_predicted[0], 1920)
df_not_null['Rage-Net_Y'] = np.multiply(ds_normalized_predicted[1], 1080)

columns_to_drop = ['Rage-Net_X', 'Rage-Net_Y']

# Drop columns so there will be no duplicates after merge
df = df.drop(columns_to_drop, axis=1, errors='ignore')

# Add to whole dataframe
df = pd.merge(df, df_not_null[['Right_eye_normalized', 'Rage-Net_X', 'Rage-Net_Y']],
              on='Right_eye_normalized', how='left')

In [9]:
df.head()

,Participant,Recording,Case,Stimul,Timestamp,Fixation_X,Fixation_Y,File,Right_eye_normalized,Left_eye_normalized,Rage-Net_X,Rage-Net_Y
0,P2,Recording5,case_1,Top center,192650117,955.0,142.0,0.jpg,0_right_eye.jpg,0_left_eye.jpg,992.763062,187.248215
1,P2,Recording5,case_1,Bottom left,194186638,151.0,972.0,1.jpg,1_right_eye.jpg,1_left_eye.jpg,89.265305,973.636658
2,P2,Recording5,case_1,Top right,195679731,1748.0,180.0,2.jpg,2_right_eye.jpg,2_left_eye.jpg,1765.614014,188.490295
3,P2,Recording5,case_1,Bottom center,197266136,942.0,985.0,3.jpg,3_right_eye.jpg,3_left_eye.jpg,924.300232,977.150940
4,P2,Recording5,case_1,Bottom right,198975859,1713.0,977.0,4.jpg,4_right_eye.jpg,4_left_eye.jpg,1726.082520,1041.627808


### Save dataframe

In [10]:
# Save dataframe
df.to_csv(data_directory, sep='\t', index=False)